## Number 2

In [7]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import time

web = "https://quotes.toscrape.com"
service = Service(executable_path="chromedriver.exe")
driver = webdriver.Chrome(service=service)
driver.get(web)


def scrape_authors():
    products = driver.find_elements(By.CLASS_NAME, 'author-details')
    author_deats = []

    for product in products:
        title_element = product.find_element(By.XPATH, "./h3[@class='author-title']").text
        author_deats.append(title_element)
        birh_element = product.find_element(By.XPATH, ".//span[@class='author-born-date']").text
        author_deats.append(birh_element)
        nationality_element = product.find_element(By.XPATH, ".//span[@class='author-born-location']").text
        author_deats.append(nationality_element)
        desc_element = product.find_element(By.XPATH, ".//div[@class='author-description']").text
        author_deats.append(desc_element)
        
    return author_deats

all_authors = []
quote_elements = driver.find_elements(By.XPATH, '//div[@class="quote"]')

num = 5
maxi = min(len(quote_elements),num)
# Iterate over each quote element

for i in range(maxi):
    quote = quote_elements[i]
    try:
        # Find and click the author link
        author_link = quote.find_element(By.XPATH, './/a[contains(@href, "/author/")]')
        author_link.click()
        
        # Wait for the author page to load
        time.sleep(2)  # Adjust if necessary
        
        # Extract author details
        
        all_authors.append(scrape_authors())
        # Navigate back to the main quotes page
        driver.back()
        
        # Wait for the quotes page to load
        time.sleep(2)  # Adjust if necessary

        # Refresh the list of quote elements to avoid stale element reference
        quote_elements = driver.find_elements(By.XPATH, '//div[@class="quote"]')
    except Exception as e:
        print(f"An error occurred: {e}")
        continue

# Close the WebDriver
driver.quit()

In [8]:
all_authors


[['Albert Einstein',
  'March 14, 1879',
  'in Ulm, Germany',
  'In 1879, Albert Einstein was born in Ulm, Germany. He completed his Ph.D. at the University of Zurich by 1909. His 1905 paper explaining the photoelectric effect, the basis of electronics, earned him the Nobel Prize in 1921. His first paper on Special Relativity Theory, also published in 1905, changed the world. After the rise of the Nazi party, Einstein made Princeton his permanent home, becoming a U.S. citizen in 1940. Einstein, a pacifist during World War I, stayed a firm proponent of social justice and responsibility. He chaired the Emergency Committee of Atomic Scientists, which organized to alert the public to the dangers of atomic warfare.At a symposium, he advised: "In their struggle for the ethical good, teachers of religion must have the stature to give up the doctrine of a personal God, that is, give up that source of fear and hope which in the past placed such vast power in the hands of priests. In their labor

### Number 1

In [55]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# Initialize the driver (e.g., Chrome)
driver = webdriver.Chrome()

# Open the target URL
driver.get('https://books.toscrape.com/')


# Wait for the product elements to be present


def extracts():
    Name = driver.find_element(By.XPATH, ".//div[@class='col-sm-6 product_main']/h1").text
    Category = driver.find_element(By.XPATH, './/ul[@class="breadcrumb"]/li[last()-1]/a').text
    price = driver.find_element(By.XPATH, ".//table[@class='table table-striped']/tbody/tr[last()-3]/td[last()]").text
    stock = driver.find_element(By.XPATH, "//table[@class='table table-striped']/tbody/tr[last()-1]/td[last()]").text

    star_rating_element = driver.find_element(By.XPATH, "//p[contains(@class, 'star-rating')]")
    star_rating = star_rating_element.get_attribute("class").split()[-1]

    desc = driver.find_element(By.XPATH, "//div[@id='product_description']/following-sibling::p").text

    ## product information
    ups = driver.find_element(By.XPATH, ".//table[@class='table table-striped']/tbody/tr[last()-6]/td[last()]").text
    
    Type = driver.find_element(By.XPATH, ".//table[@class='table table-striped']/tbody/tr[last()-5]/td[last()]").text

    price_ex = driver.find_element(By.XPATH, ".//table[@class='table table-striped']/tbody/tr[last()-4]/td[last()]").text

    price_in = driver.find_element(By.XPATH, ".//table[@class='table table-striped']/tbody/tr[last()-3]/td[last()]").text

    tax = driver.find_element(By.XPATH, ".//table[@class='table table-striped']/tbody/tr[last()-2]/td[last()]").text

    avaliablity = driver.find_element(By.XPATH, ".//table[@class='table table-striped']/tbody/tr[last()-1]/td[last()]").text

    review = driver.find_element(By.XPATH, ".//table[@class='table table-striped']/tbody/tr[last()]/td[last()]").text

    Bok_list = [Name,Category,price,stock,star_rating,desc, ups, Type, price_ex,price_in, tax ,avaliablity,review]


    return Bok_list

def page_extract():
    wait = WebDriverWait(driver, 10)
    wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'product_pod')))
    products = driver.find_elements(By.CLASS_NAME, 'product_pod')
    # Find all product elements
    all_books = []

    # Ensure products is a list of WebElements
    if isinstance(products, list) and all(isinstance(product, webdriver.remote.webelement.WebElement) for product in products):
        max_products = min(len(products), 20)  # Avoid IndexError if fewer than 20 elements
        
        for i in range(max_products):
            try:
                # Re-locate the products list in each iteration to handle dynamic page updates
                products = driver.find_elements(By.CLASS_NAME, 'product_pod')
                
                # Ensure the list has enough elements to avoid IndexError
                if i >= len(products):
                    break
                
                book = products[i]
                book_link = book.find_element(By.XPATH, ".//h3//a")
                book_link.click()
                
                all_books.append(extracts())

                # Navigate back to the product list page
                driver.back()
                
                # Wait for the product list to be present again
                wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'product_pod')))
                
            except Exception as e:
                print(f"An error occurred at index {i}: {e}")
    else:
        print("The 'products' variable is not a list of WebElements.")

    return all_books


def pages_extract(num):
    current_page = 0
    num_pages_to_scrape = num
    all_bok = []
    while current_page <= num_pages_to_scrape:
        try:
            x = page_extract()
            all_bok.append(x)
            #Here we click the next button and wait a bit for page to load properly
            next_button = driver.find_element(By.XPATH, '//li[@class="next"]/a')
            next_button.click()
            time.sleep(2)

            current_page += 1
        except:
            break
    return all_bok

y = pages_extract(4)

print(y)

driver.quit()




3


In [ ]:
print(y)


### Number 3

In [57]:
import requests
from bs4 import BeautifulSoup
import random

def fetch_random_wikipedia_page():
    try:
        # URL for a random Wikipedia page
        random_url = "https://en.wikipedia.org/wiki/Special:Random"
        
        # Send a GET request to the random URL
        response = requests.get(random_url)
        
        # Check if the request was successful
        if response.status_code != 200:
            raise Exception(f"Failed to fetch page, status code: {response.status_code}")
        
        # Parse the page content
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Extract the title of the page
        title = soup.find('h1', {'id': 'firstHeading'}).text
        
        # Extract the first paragraph of the page
        first_paragraph = soup.find('p').text
        
        # Print the title and first paragraph
        print(f"Title: {title}\n")
        print(f"First paragraph: {first_paragraph}\n")
        
        # Optionally, return the data
        return {
            'title': title,
            'first_paragraph': first_paragraph,
            'url': response.url
        }
        
    except Exception as e:
        print(f"An error occurred: {e}")

# Fetch and display a random Wikipedia page
page_data = fetch_random_wikipedia_page()

# If needed, do something with the data
if page_data:
    print(f"Scraped data from: {page_data['url']}")

Title: Dance of the Wicked

First paragraph: Dance of the Wicked is heavy metal band Sister Sin's first studio album, released 2003. Recorded and produced in 2003 by Jan Jan at Triton Studios Gothenburg. Released and Distributed by Sleaszy Rider Records & EMI Greece Distribution.


Scraped data from: https://en.wikipedia.org/wiki/Dance_of_the_Wicked
